In [1]:
import numpy as np
import matplotlib as mpl
mpl.use("TkAgg")
import matplotlib.pyplot as plt

data = np.load("isotropic1024_slice.npz")
u = data["u"]
v = data["v"]
w = data["w"]

# part-1 ---> calculating the kolmogorv stats:
Lx = 2 * np.pi # size of the domain
Ly = 2 * np.pi
Nx = 1024 # no. of grid points in one direction
Ny = 1024
dx = Lx/Nx # size of grid cell
dy = Ly/Ny
l = 1.364 # integral length scale 
nu = 0.000185
rms_u = 0
rms_v = 0
rms_w = 0
pts = 1024 * 1024 # represents total grid pts

for i in range(1024):
    for j in range(1024):
        rms_u = rms_u + (u[i][j])*(u[i][j])
        rms_v = rms_v + (v[i][j])*(v[i][j])
        rms_w = rms_w + (w[i][j])*(w[i][j])

rms_u = np.sqrt(rms_u/pts)
rms_v = np.sqrt(rms_v/pts)
rms_w = np.sqrt(rms_w/pts)

U = rms_u;

Re = (U * l)/nu; # Reynolds number

epsilon = (U ** 3)/l
eta = ((nu ** 3)/epsilon) ** 0.25 # kolmogorov length scale
tau_eta = (nu / epsilon) ** 0.5 # kolmogorov time scale
u_eta = (nu * epsilon) ** 0.25 # kolmogorov velocity scale
Re_eta = (u_eta * eta)/nu # this should be one
# print(Re_eta)

ratio = dx/eta

print("Reynolds Number : " + str(Re))
print("Kolmogorov Length Scale : " + str(eta))
print("Kolmogorov Time Scale : " + str(tau_eta))
print("Kolmogorov Velocity Scale : " + str(u_eta))
print("Ratio of grid cell size and the Kolmogorov length :" + str(ratio))

Reynolds Number : 4477.861008221851
Kolmogorov Length Scale : 0.0024917898561977753
Kolmogorov Time Scale : 0.03356225236459529
Kolmogorov Velocity Scale : 0.0742438209786646
Ratio of grid cell size and the Kolmogorov length :2.4624561081187544


In [2]:
# part-2

KE = np.zeros((1024,1024))
avg_KE = 0

for i in range(1024):
    for j in range(1024):
        KE[i][j] = 0.5 * (u[i][j] ** 2 + v[i][j] ** 2 + w[i][j] ** 2)
        avg_KE = avg_KE + KE[i][j]
avg_KE = (avg_KE/pts)

normalised_KE = (1/avg_KE) * (KE);

plt.figure()
KE_colorbar = plt.pcolor(normalised_KE[::,::] )
plt.title("Kinetic Energy Field")
plt.colorbar(KE_colorbar)
plt.show()




In [3]:
# part-3
dv_dy = np.zeros((1024,1024));
du_dy = np.zeros((1024,1024));
for j in range(1024):
    for i in range(1024):
        if i == 0:
            du_dy[i][j] = (u[i+1][j] - u[i][j])/dy
            dv_dy[i][j] = (v[i+1][j] - v[i][j])/dy
        elif i == 1023:
            du_dy[i][j] = (u[i][j] - u[i-1][j])/dy
            dv_dy[i][j] = (v[i][j] - v[i-1][j])/dy
        else:
            du_dy[i][j] = (u[i+1][j] - u[i-1][j])/(2 * dy)
            dv_dy[i][j] = (v[i+1][j] - v[i-1][j])/(2 * dy)

dv_dx = np.zeros((1024,1024));
du_dx = np.zeros((1024,1024));
for i in range(1024):
    for j in range(1024):
        if j == 0:
            du_dx[i][j] = (u[i][j+1] - u[i][j])/dx
            dv_dx[i][j] = (v[i][j+1] - v[i][j])/dx
        elif j == 1023:
            du_dx[i][j] = (u[i][j] - u[i][j-1])/dx
            dv_dx[i][j] = (v[i][j] - v[i][j-1])/dx
        else:
            du_dx[i][j] = (u[i][j+1] - u[i][j-1])/(2 * dx)
            dv_dx[i][j] = (v[i][j+1] - v[i][j-1])/(2 * dx)

vorticity_z = np.zeros((1024,1024))
for i in range(1024):
    for j in range(1024):
        vorticity_z[i][j] = dv_dx[i][j] - du_dy[i][j]

rms_vorticity_z = 0;
for i in range(1024):
    for j in range(1024):
        rms_vorticity_z = rms_vorticity_z + (vorticity_z[i][j] ** 2);

rms_vorticity_z = (rms_vorticity_z/pts) ** 0.5
normalised_vorticity_z = (1/rms_vorticity_z) * (vorticity_z);

plt.figure()
vorticity_z_colorbar = plt.pcolor(normalised_vorticity_z[::,::])
plt.title("Vorticity Field")
plt.colorbar(vorticity_z_colorbar)
plt.show()

In [4]:
# part-4
    

# this function makes the normalised pdf and cdf of the quantity of interest

def make_pdf_cdf(arr,num_bins):
    mean_arr = np.mean(arr);
    #print(mean_arr)
    sd_arr = np.std(arr,ddof = 1); # arr standard deviation
    #print(sd_arr)
    normalised_arr = (arr - mean_arr) / sd_arr
    #print(normalised_arr)

    hist, bins = np.histogram(normalised_arr.ravel(), bins = num_bins)
    bin_width = bins[1] - bins[0]
    bin_centres = np.zeros((num_bins))
    #print(bin_centres)
    for i in range(num_bins):
        bin_centres[i] = (bins[i] + bins[i+1])/2
    arr_pdf = hist / (np.sum(hist) * bin_width)
    #print(hist)
    arr_cdf = [np.sum(arr_pdf[:i]) for i in range (len(hist))]
    for i in range(num_bins):
        arr_cdf[i] = arr_cdf[i]/arr_cdf[num_bins-1] # normalizing the u_cdf

    return bin_centres,arr_pdf,arr_cdf




def kurtosis_calculator(arr):
    mean_arr = np.mean(arr);
    sd_arr = np.std(arr,ddof = 1);
    kurtosis_val = 0
    rows = arr.shape[0];
    cols = arr.shape[1];
    for i in range(rows):
        for j in range(cols):
            kurtosis_val = kurtosis_val + (arr[i][j] - mean_arr) ** 4;
    kurtosis_val = kurtosis_val/((sd_arr ** 4) * pts)
    return kurtosis_val  
    

def skewness_calculator(arr):
    mean_arr = np.mean(arr);
    sd_arr = np.std(arr,ddof = 1);
    skewness_val = 0
    rows = arr.shape[0];
    cols = arr.shape[1];
    for i in range(rows):
        for j in range(cols):
            skewness_val = skewness_val + (arr[i][j] - mean_arr) ** 3;
    skewness_val = skewness_val/((sd_arr ** 3) * pts)
    return skewness_val  


In [6]:
# part - 4(a)


num_bin = 50;


# making the normal distribution and its cdf

x = np.linspace(-4, 4, num_bin)
mu, sigma = 0, 1
normal_pdf = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
normal_cdf = [np.sum(normal_pdf[:i]) for i in range(num_bin)]
for i in range(num_bin):
    normal_cdf[i] = normal_cdf[i]/normal_cdf[num_bin-1] # normalizing the u_cdf 




bin_centres_u,u_pdf,u_cdf = make_pdf_cdf(u,num_bin)
bin_centres_v,v_pdf,v_cdf = make_pdf_cdf(v,num_bin)
bin_centres_w,w_pdf,w_cdf = make_pdf_cdf(w,num_bin)

# pdf comparision plots

plt.figure(1)
plt.plot(bin_centres_u, u_pdf, label = "u - Distribution", linewidth = 2)
plt.plot(bin_centres_v, v_pdf, label = "v - Distribution", linewidth = 2)
plt.plot(bin_centres_w, w_pdf, label = "w - Distribution", linewidth = 2)
plt.plot(x, normal_pdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and u,v,w - distributions")
plt.legend()
plt.show()

# cdf comparision plots

plt.figure(2)
plt.plot(bin_centres_u, u_cdf, label = "u - Distribution", linewidth = 2)
plt.plot(bin_centres_v, v_cdf, label = "v - Distribution", linewidth = 2)
plt.plot(bin_centres_w, w_cdf, label = "w - Distribution", linewidth = 2)
plt.plot(x, normal_cdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and u,v,w - distributions")
plt.legend()
plt.show()

# Skewness
skewness_u = skewness_calculator(u);
skewness_v = skewness_calculator(v);
skewness_w = skewness_calculator(w);

print("skewness_u : " + str(skewness_u))
print("skewness_v : " + str(skewness_v))
print("skewness_w : " + str(skewness_w))


# Kurtosis
kurtosis_u = kurtosis_calculator(u);
kurtosis_v = kurtosis_calculator(v);
kurtosis_w = kurtosis_calculator(w);

print("kurtosis_u : " + str(kurtosis_u))
print("kurtosis_v : " + str(kurtosis_v))
print("kurtosis_w : " + str(kurtosis_w))






skewness_u : 0.17190901669220843
skewness_v : 0.09136925992119718
skewness_w : -0.2201799985538492
kurtosis_u : 3.04540666483723
kurtosis_v : 2.5911054085216922
kurtosis_w : 2.582971778164235


In [7]:
# part - 4(b)

num_bin = 100

x = np.linspace(-15, 15, num_bin)
mu, sigma = 0, 1
normal_pdf = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
normal_cdf = [np.sum(normal_pdf[:i]) for i in range(num_bin)]
for i in range(num_bin):
    normal_cdf[i] = normal_cdf[i]/normal_cdf[num_bin-1] # normalizing the u_cdf 

bin_centres_vorticity_z,vorticity_z_pdf,vorticity_z_cdf = make_pdf_cdf(vorticity_z,num_bin)


# pdf comparision plots

plt.figure(1)
plt.plot(bin_centres_vorticity_z, vorticity_z_pdf, label = "vorticity_z - Distribution", linewidth = 2)
plt.plot(x, normal_pdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and vorticity_z - distributions")
plt.legend()
plt.show()

# cdf comparision plots

plt.figure(2)
plt.plot(bin_centres_vorticity_z, vorticity_z_cdf, label = "vorticity_z - Distribution", linewidth = 2)
plt.plot(x, normal_cdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and vorticity_z - distributions")
plt.legend()
plt.show()

# Skewness
skewness_vorticity_z = skewness_calculator(vorticity_z)
print("skewness_vorticity_z : " + str(skewness_vorticity_z))

# Kurtosis
kurtosis_vorticity_z = kurtosis_calculator(vorticity_z)
print("kurtosis_vorticity_z : " + str(kurtosis_vorticity_z))

skewness_vorticity_z : 0.05660021464815684
kurtosis_vorticity_z : 7.60047927102915


In [8]:
# part - 4(c)(1)

numbin = 100

x = np.linspace(-20,20, num_bin)
mu, sigma = 0, 1
normal_pdf = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
normal_cdf = [np.sum(normal_pdf[:i]) for i in range(num_bin)]
for i in range(num_bin):
    normal_cdf[i] = normal_cdf[i]/normal_cdf[num_bin-1] # normalizing the u_cdf 


bin_centres_du_dx,du_dx_pdf,du_dx_cdf = make_pdf_cdf(du_dx,num_bin)


# pdf comparision plots

plt.figure(1)
plt.plot(bin_centres_du_dx, du_dx_pdf, label = "du_dx - Distribution", linewidth = 2)
plt.plot(x, normal_pdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and du_dx - distributions")
plt.legend()
plt.show()

#cdf comparision plots

plt.figure(2)
plt.plot(bin_centres_du_dx, du_dx_cdf, label = "du_dx - Distribution", linewidth = 2)
plt.plot(x, normal_cdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and du_dx - distributions")
plt.legend()
plt.show()

# Skewness
skewness_du_dx = skewness_calculator(du_dx)
print("skewness_du_dx : " + str(skewness_du_dx))


# Kurtosis
kurtosis_du_dx = kurtosis_calculator(du_dx)
print("kurtosis_du_dx : " + str(kurtosis_du_dx))

skewness_du_dx : -0.33346105810277066
kurtosis_du_dx : 10.806434588569548


In [9]:
# part - 4(c)(2)

x = np.linspace(-20,20, num_bin)
mu, sigma = 0, 1
normal_pdf = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
normal_cdf = [np.sum(normal_pdf[:i]) for i in range(num_bin)]
for i in range(num_bin):
    normal_cdf[i] = normal_cdf[i]/normal_cdf[num_bin-1] # normalizing the u_cdf 


bin_centres_dv_dy,dv_dy_pdf,dv_dy_cdf = make_pdf_cdf(dv_dy,num_bin)


# pdf comparision plots

plt.figure(1)
plt.plot(bin_centres_dv_dy, dv_dy_pdf, label = "dv_dy - Distribution", linewidth = 2)
plt.plot(x, normal_pdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and dv_dy - distributions")
plt.legend()
plt.show()

# cdf comparision plots

plt.figure(2)
plt.plot(bin_centres_dv_dy, dv_dy_cdf, label = "dv_dy - Distribution", linewidth = 2)
plt.plot(x, normal_cdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and dv_dy - distributions")
plt.legend()
plt.show()

# Skewness
skewness_dv_dy = skewness_calculator(dv_dy)
print("skewness_dv_dy : " + str(skewness_dv_dy))


# Kurtosis
kurtosis_dv_dy = kurtosis_calculator(dv_dy)
print("kurtosis_dv_dy : " + str(kurtosis_dv_dy))


skewness_dv_dy : 0.11653157769132681
kurtosis_dv_dy : 10.578385110472853


In [10]:
# part - 4(d)

num_bin = 100

entrosphy = np.zeros((1024,1024));
for i in range(1024):
    for j in range(1024):
        entrosphy[i][j] = vorticity_z[i][j] ** 2;


x = np.linspace(-17,17, num_bin)
mu, sigma = 0, 1
normal_pdf = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
normal_cdf = [np.sum(normal_pdf[:i]) for i in range(num_bin)]
for i in range(num_bin):
    normal_cdf[i] = normal_cdf[i]/normal_cdf[num_bin-1] # normalizing the u_cdf 


bin_centres_entrosphy,entrosphy_pdf,entrosphy_cdf = make_pdf_cdf(entrosphy,num_bin)

print(bin_centres_entrosphy[1])


# pdf comparision plots

plt.figure(1)
plt.plot(bin_centres_entrosphy, entrosphy_pdf, label = "entrosphy - Distribution", linewidth = 2)
plt.plot(x, normal_pdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and entrosphy - distributions")
plt.legend()
plt.show()

# cdf comparision plots

plt.figure(2)
plt.plot(bin_centres_entrosphy, entrosphy_cdf, label = "entrosphy - Distribution", linewidth = 2)
plt.plot(x, normal_cdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and entrosphy - distributions")
plt.legend()
plt.show()

# Skewness
skewness_entrosphy = skewness_calculator(entrosphy)
print("skewness_entrosphy : " + str(skewness_entrosphy))


# Kurtosis
kurtosis_entrosphy = kurtosis_calculator(entrosphy)
print("kurtosis_entrosphy : " + str(kurtosis_entrosphy))

0.8923273178883141
skewness_entrosphy : 12.962020450273679
kurtosis_entrosphy : 435.07139695472176


In [11]:
# part - 4(e)

num_bin = 100

x = np.linspace(-2.5,2.5, num_bin)
mu, sigma = 0, 1
normal_pdf = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
normal_cdf = [np.sum(normal_pdf[:i]) for i in range(num_bin)]
for i in range(num_bin):
    normal_cdf[i] = normal_cdf[i]/normal_cdf[num_bin-1] # normalizing the u_cdf 



u1 = u[::,0::128]
u2 = u[::,129::256]
u3 = u[::,257::512]

bin_centres_u1,u1_pdf,u1_cdf = make_pdf_cdf(u1,num_bin)
bin_centres_u2,u2_pdf,u2_cdf = make_pdf_cdf(u2,num_bin)
bin_centres_u3,u3_pdf,u3_cdf = make_pdf_cdf(u3,num_bin)

# pdf comparision plots

plt.figure(1)
plt.plot(bin_centres_u1, u1_pdf, label = "u1 - Distribution", linewidth = 2)
plt.plot(bin_centres_u2, u2_pdf, label = "u2 - Distribution", linewidth = 2)
plt.plot(bin_centres_u3, u3_pdf, label = "u3 - Distribution", linewidth = 2)
plt.plot(x, normal_pdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and u1,u2,u3 - distributions")
plt.legend()
plt.show()

# cdf comparision plots

plt.figure(2)
plt.plot(bin_centres_u1, u1_cdf, label = "u1 - Distribution", linewidth = 2)
plt.plot(bin_centres_u2, u2_cdf, label = "u2 - Distribution", linewidth = 2)
plt.plot(bin_centres_u3, u3_cdf, label = "u3 - Distribution", linewidth = 2)
plt.plot(x, normal_cdf, label = "Normal Distribution", linewidth = 2)
plt.yscale("log")
plt.title("Comparison of Normal Distribution and u1,u2,u3 - distributions")
plt.legend()
plt.show()

# Skewness
skewness_u1 = skewness_calculator(u1);
skewness_u2 = skewness_calculator(u2);
skewness_u3 = skewness_calculator(u3);
print("skewness_u1 : " + str(skewness_u1))
print("skewness_u2 : " + str(skewness_u2))
print("skewness_u3 : " + str(skewness_u3))



# Kurtosis
kurtosis_u1 = kurtosis_calculator(u1);
kurtosis_u2 = kurtosis_calculator(u2);
kurtosis_u3 = kurtosis_calculator(u3);
print("kurtosis_u1 : " + str(kurtosis_u1))
print("kurtosis_u2 : " + str(kurtosis_u2))
print("kurtosis_u3 : " + str(kurtosis_u3))





skewness_u1 : 0.0005713057618805946
skewness_u2 : 0.000830360526160265
skewness_u3 : -0.00028853409546430135
kurtosis_u1 : 0.022610235662881676
kurtosis_u2 : 0.01449052620623452
kurtosis_u3 : 0.004277477313540519


In [13]:
# part - 5

Nlin = np.zeros((1024,1024)) # Non - linear term
Visc = np.zeros((1024,1024)) # Viscous term
for i in range(1024):
    for j in range(1024):
        Nlin[i][j] = u[i][j] * du_dx[i][j] + v[i][j] * du_dy[i][j]
d2u_dx2 = np.zeros((1024,1024))
d2u_dy2 = np.zeros((1024,1024))
d2v_dx2 = np.zeros((1024,1024))
d2v_dy2 = np.zeros((1024,1024))

for j in range(1024):
    for i in range(1024):
        if i == 0:
            d2u_dy2[i][j] = (du_dy[i+1][j] - du_dy[i][j])/dy
            d2v_dy2[i][j] = (dv_dy[i+1][j] - dv_dy[i][j])/dy
        elif i == 1023:
            d2u_dy2[i][j] = (du_dy[i][j] - du_dy[i-1][j])/dy
            d2v_dy2[i][j] = (dv_dy[i][j] - dv_dy[i-1][j])/dy
        else:
            d2u_dy2[i][j] = (du_dy[i+1][j] - du_dy[i-1][j])/(2 * dy)
            d2v_dy2[i][j] = (dv_dy[i+1][j] - dv_dy[i-1][j])/(2 * dy)


for i in range(1024):
    for j in range(1024):
        if j == 0:
            d2u_dx2[i][j] = (du_dx[i][j+1] - du_dx[i][j])/dx
            d2v_dx2[i][j] = (dv_dx[i][j+1] - dv_dx[i][j])/dx
        elif j == 1023:
            d2u_dx2[i][j] = (du_dx[i][j] - du_dx[i][j-1])/dx
            d2v_dx2[i][j] = (dv_dx[i][j] - dv_dx[i][j-1])/dx
        else:
            d2u_dx2[i][j] = (du_dx[i][j+1] - du_dx[i][j-1])/(2 * dx)
            d2v_dx2[i][j] = (dv_dx[i][j+1] - dv_dx[i][j-1])/(2 * dx)


for i in range(1024):
    for j in range(1024):
        Visc[i][j] = nu * (d2u_dx2[i][j] + d2u_dy2[i][j]);

        

# plotting the two terms:

plt.figure()
Nlin_colorbar = plt.pcolor(Nlin[::,::])
plt.colorbar(Nlin_colorbar)
plt.show()

plt.figure()
Visc_colorbar = plt.pcolor(Visc[::,::])
plt.colorbar(Visc_colorbar)
plt.show()




# LocaL Reynolds Number:

Re_local = np.zeros((1024,1024))
for i in range(1024):
    for j in range(1024):
            if Visc[i][j] != 0:
                Re_local[i][j] = np.abs(Nlin[i][j]/Visc[i][j])
            else:
                Re_local[i][j] = 10 ** -20

Re_local_clipped = np.clip(Re_local, 1e-4, 1e7)  # I have Limited max to 10^7 as only 5 points out of 1024 * 1024 have re_local > 10 ^ 7
plt.figure()
cmap = plt.pcolor(np.log(Re_local_clipped), vmin=-4, vmax=7)
plt.colorbar(cmap)
plt.title("Local Reynolds Number field (Log to the base 10 scale)")
plt.show()
